In [0]:
%sql
-- Definição do catálogo e schema utilizados na camada Silver.

USE CATALOG mvp;
USE SCHEMA silver;

In [0]:
# Imports necessários para definição de schema e criação do DataFrame manual de companhias aéreas

from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql import Row

In [0]:
# Criação manual da dimensão de companhias aéreas domésticas pois não existe este dado disponível na base da ANAC
# Definimos a utilização destas 14 após a consulta realizada no notebook mvp04.silver.voos
# Definimos a operação como 'Doméstica Mista' devido ao filtro realizado no notebook mvp04.silver.voos conforme objetivo do projeto
# Colocamos uma flag definida como 1 para controle de companhia ativas na base
dados_companhias = [
    ("AZU", "Azul Linhas Aéreas", "Doméstica Mista", 1),
    ("TAM", "LATAM Airlines", "Doméstica Mista", 1),
    ("GLO", "Gol Linhas Aéreas", "Doméstica Mista", 1),
    ("ACN", "Azul Conecta", "Doméstica Mista", 1),
    ("PTB", "Passaredo Transportes Aéreos", "Doméstica Mista", 1),
    ("PAM", "MAP Linhas Aéreas", "Doméstica Mista", 1),
    ("ABJ", "Abaeté Aviação", "Doméstica Mista", 1),
    ("SID", "Sideral Linhas Aéreas", "Doméstica Mista", 1),
    ("CQB", "Apuí Táxi Aéreo", "Doméstica Mista", 1),
    ("MWM", "Modern Logistics", "Doméstica Mista", 1),
    ("TOT", "Total Express", "Doméstica Mista", 1),
    ("TTL", "Total Linhas Aéreas", "Doméstica Mista", 1),
    ("OMI", "Omni Táxi Aéreo", "Doméstica Mista", 1),
    ("LTG", "LATAM Cargo Brasil", "Doméstica Mista", 1)
]

In [0]:
# Definição do schema estruturado do DataFrame Silver companhias

schema_companhias = StructType([
    StructField("icao_empresa_aerea", StringType(), False),
    StructField("nome_companhia", StringType(), False),
    StructField("tipo_operacao", StringType(), False),
    StructField("ativo_analise", IntegerType(), False)
])

In [0]:
# Criação do DataFrame Silver usando os dados e o schema definidos

df_companhias_silver = spark.createDataFrame(
    dados_companhias,
    schema=schema_companhias
)

In [0]:
# Criação da tabela Silver em formato delta para armazenar os dados

(
    df_companhias_silver
    .write
    .format("delta")
    .mode("overwrite")
    .option("mergeSchema", "true")
    .saveAsTable("mvp.silver.companhias")
)

In [0]:
%sql
-- Validação da dimensão de companhias

SELECT *
FROM mvp.silver.companhias
ORDER BY icao_empresa_aerea;

icao_empresa_aerea,nome_companhia,tipo_operacao,ativo_analise
ABJ,Abaeté Aviação,Doméstica Mista,1
ACN,Azul Conecta,Doméstica Mista,1
AZU,Azul Linhas Aéreas,Doméstica Mista,1
CQB,Apuí Táxi Aéreo,Doméstica Mista,1
GLO,Gol Linhas Aéreas,Doméstica Mista,1
LTG,LATAM Cargo Brasil,Doméstica Mista,1
MWM,Modern Logistics,Doméstica Mista,1
OMI,Omni Táxi Aéreo,Doméstica Mista,1
PAM,MAP Linhas Aéreas,Doméstica Mista,1
PTB,Passaredo Transportes Aéreos,Doméstica Mista,1


In [0]:
%sql
SELECT COUNT(*) AS total_companhias
FROM mvp.silver.companhias;

total_companhias
14
